In [ ]:
import pandas as pd
import csv
import os
import io
from zipfile import ZipFile,ZipInfo
import editdistance
from collections import Counter
import numpy as np
from multiprocessing import Pool
import multiprocessing as mp
import unicodedata
import re
import spacy
#spacy.prefer_gpu()
#import contextualSpellCheck

nlp = spacy.load('en_core_web_sm')
nlp.max_length=10_000_000
#contextualSpellCheck.add_to_pipe(nlp)
#from spacy_hunspell import spaCyHunSpell

In [ ]:
def editer(row):
    return editdistance.distance(row['before'],row['after'])

In [ ]:
def extract_zip(input_zip):
    input_zip=ZipFile(input_zip)
    return {name: input_zip.read(name).decode('utf-8').strip() for name in input_zip.namelist()}

In [ ]:
#import torch
#roberta = torch.hub.load('pytorch/fairseq', 'roberta.large')
#roberta.eval()

In [ ]:
def docenizer(dict_content):

    str1=re.sub("@@.*|<P>|/\S*/","",dict_content)
    str2 = re.sub(' +',' ',str1)
    str2=str2.replace("@ @ @ @ @ @ @ @ @ @","@@@@@@@@@@")
    str2=str2.replace("\n\n","")
    doc=nlp(str2)
    to_hold=[]
    n_words=0
    n_sents=0
    for sent in doc.sents:
        if len(sent)==1:
            continue
        cur_sent=[]
        for word in sent:
            cur_sent.append(word.text)
            n_words+=1
        to_hold.append(' '.join(cur_sent))
        n_sents+=1
    content='\n'.join(to_hold)
    content=content.lower()
    #content=content.replace("@@@@@@@@@@", "<mask>")
    return content,n_words,n_sents

In [ ]:
def vocab_maker(dict_content):
    token_list=[]
    pos_list=[]
    lemma_list=[]
    str1=re.sub("@@.*|<P>|/\S*/","",dict_content)
    str2 = re.sub(' +',' ',str1)
    str2=str2.replace("@ @ @ @ @ @ @ @ @ @","@@@@@@@@@@")
    str2=str2.replace("\n\n","")
    doc=nlp(str2)
    for token in doc:
        token_list.append(token.text)
        pos_list.append(token.pos_)
        lemma_list.append(token.lemma_)
    return token_list,pos_list,lemma_list

In [ ]:
def write_to_file(fnames,dec,set_type):
    save_file="./"+str(dec)+"/"+set_type+".txt"
    print(save_file)
    with open(save_file,'w') as f:
        for doc in fnames:
            f.write(zipfiles[doc]+"\n\n")

## Read data

In [ ]:
_dir = "/resources/corpora/COHA/text/"
#_dir = "/resources/corpora/COHA/CCOHA/tagged/"
#_dir = "/resources/corpora/COHA/ALL/"
files = sorted(os.listdir(_dir))

to_keep=[]
for f in files:
    if 'zip' in f:
        to_keep.append(f)
len(to_keep)

In [ ]:
to_process=to_keep[0]

In [ ]:
zipfiles=extract_zip(os.path.join(_dir, to_process))
zfnames=list(zipfiles.keys())
docs=list(zipfiles.values())

In [ ]:
curr_year=to_process.split('_')[1][:-1]
curr_year

In [ ]:
n_proc = mp.cpu_count()-1

pool = Pool(n_proc)
results=pool.map_async(vocab_maker,docs)
pool.close()
pool.join()

In [ ]:
results=results.get()
token_list = [val[0] for val in results]
pos_list=[val[1] for val in results]
lemma_list=[val[2] for val in results]

In [ ]:
full_token_list = [item for sublist in token_list for item in sublist]
len(full_token_list)

In [ ]:
n_proc = mp.cpu_count()-1

pool = Pool(n_proc)
results=pool.map_async(docenizer,docs)
pool.close()
pool.join()

results=results.get()
contents=[val[0] for val in results]
n_words=[val[1] for val in results]
n_sents=[val[2] for val in results]

In [ ]:
zipfiles=dict(zip(zfnames,contents))

In [ ]:
def setmaker(fname,curr_year):
    cur_set=zipfiles[fname].split('\n')
    testset=cur_set[:1000]
    validset=cur_set[1001:2001]
    
    test_file="./"+str(curr_year)+"/test.txt"
    print(test_file)
    with open(test_file,'w') as f:
        for sent in testset:
            f.write(sent+"\n\n")

    valid_file="./"+str(curr_year)+"/valid.txt"           
    with open(valid_file,'w') as f:
        for sent in testset:
            f.write(sent+"\n\n") 
    return '\n'.join(cur_set[2002:])

In [ ]:
genres=pd.DataFrame({'fname':zfnames,'content':contents,'n_words':n_words,'n_sents':n_sents})
genres['gtype']=genres['fname'].str.split('_').str[0]
genres

In [ ]:
min_docs=genres.sort_values('n_sents', ascending=False).head(1)['fname'].to_list()
min_docs[0]

In [ ]:
zipfiles[min_docs[0]]=setmaker(min_docs[0],curr_year)

In [ ]:
trainset_list=zipfiles.keys()

all_dfs=pd.concat(dfs)
all_dfs.columns=['token','lemma','pos','dep','shape','is_alpha','is_stop']
all_dfs=all_dfs.loc[~(all_dfs.pos=='SPACE')]
all_dfs.token=all_dfs.token.str.lower()
#all_dfs.index[all_dfs['pos'] == 'PUNCT']
all_dfs.reset_index(inplace=True,drop=True)
#all_dfs.loc[all_dfs.is_stop==True]['lemma'].value_counts().head(30)
punct_list=all_dfs.index[(all_dfs['pos'] == 'PUNCT') & (all_dfs['token'] == '.')]
punct_list+=1
all_token=all_dfs.token.to_list()
len(punct_list)
sents=np.split(all_token,punct_list)

In [ ]:
write_to_file(trainset_list,curr_year,'train')